# Customer Churn Prediction 
Machine Learning Project - Decison Tree Classification 

Authors:
* José Marcos Leal Barbosa Filho
* Lucas Ismael Campos Medeiros

Institution: Universidade Federal do Rio Grande do Norte - Brazil.

## Dataset Description

It is a **customer churn** modeling dataset containing 10.000 rows (each representing an unique customer) with 14 columns: 13 general features and one target feature (**Exited**). The data is composed of both numerical and categorical features:

**Numeric Features:**

    RowNumber: The sequence number of the rows. 
    CustomerId: A unique ID of the customer.
    CreditScore: The credit score of the customer,
    Age: The age of the customer,
    Tenure: The number of months the client has been with the firm.
    Balance: Balance remaining in the customer account,
    NumOfProducts: The number of products purchased by the customer.
    EstimatedSalary: The estimated salary of the customer.

**Categorical Features:**

    Surname: The surname of the customer.
    Geography: The country of the customer.
    Gender: M/F
    HasCrCard: Whether the customer has a credit card or not.
    IsActiveMember: Whether the customer is active or not.

**The target column:** 

    Exited — Whether the customer churned or not.

The dataset can be seen and downloaded [here](https://www.kaggle.com/datasets/aakash50897/churn-modellingcsv?resource=download).

## Load Libraries

In [ ]:
%%capture
!pip install wandb
#!pip install wandb==0.10.17
#!pip install pytest pytest-sugar
#!pip install pandas-profiling==3.1.0
!pip install tensorflow_addons

In [ ]:
import wandb
import logging
import tempfile
import os
import joblib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import h5py
import time
import datetime
import pytz
import IPython
import tensorflow_addons as tfa
import tensorflow as tf
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from wandb.keras import WandbCallback
from keras.callbacks import EarlyStopping


## Login to wandb

In [ ]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 1 - Extract, Transform and Load (ETL)

### 1.1 Fetch Data

In [ ]:
# columns used
columns = ['RowNumber', 'CustomerId', 'Surname', 'CreditScore',
           'Geography', 'Gender', 'Age', 'Tenure', 
           'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
           'EstimatedSalary', 'Exited']
# importing the dataset
churndf = pd.read_csv("https://github.com/x4nth055/pythoncode-tutorials/raw/master/machine-learning/customer-churn-detection/Churn_Modelling.csv",
                      header=None,
                      names=columns)
churndf.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,1,15634602,Hargrave,619,France,Female,42,2,0,1,1,1,101348.88,1
2,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,3,15619304,Onio,502,France,Female,42,8,159660.8,3,1,0,113931.57,1
4,4,15701354,Boni,699,France,Female,39,1,0,2,0,0,93826.63,0


* It was chosen to remove the following columns:

  * **RowNumber:** Indicates only the sequence number of the lines;
  * **CustomerId:** High cardinality column with 10,000 unique IDs;
  * **Surname:** Column with high cardinality, showing the last names of each consumer.

In [ ]:
# removing unecessary columns and reseting indexes
churndf = churndf.drop([0,])
churndf.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)
churndf.reset_index(drop=True,inplace=True)
churndf.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.8,3,1,0,113931.57,1
3,699,France,Female,39,1,0,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.1,0


In [ ]:
# saving to csv
churndf.to_csv("raw_data.csv", index=False)

In [ ]:
# Saving artifact to wandb
!wandb artifact put \
       --name churn_prediction_project_nn/raw_data.csv \
       --type raw_data \
       --description "Customer Churn NN" raw_data.csv

wandb: Uploading file raw_data.csv to: "eec1509/churn_prediction_project_nn/raw_data.csv:latest" (raw_data)
wandb: Currently logged in as: macleal (eec1509). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.12.21
wandb: Run data is saved locally in /content/wandb/run-20220714_121421-9syst0ri
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run celestial-sun-8
wandb: ⭐️ View project at https://wandb.ai/eec1509/churn_prediction_project_nn
wandb: 🚀 View run at https://wandb.ai/eec1509/churn_prediction_project_nn/runs/9syst0ri
Artifact uploaded, use this artifact in a run by adding:

    artifact = run.use_artifact("eec1509/churn_prediction_project_nn/raw_data.csv:v0")

wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: Synced celestial-sun-8: https://wandb.ai/eec1509/churn_prediction_project_nn/runs/9syst0ri
wandb: Synced 5 W&B file(s), 0 med

### 1.2 Exploratory Data Analysis (EDA)

In [ ]:
# save_code tracking all changes of the notebook and sync with Wandb
run = wandb.init(project="churn_prediction_project_nn", save_code=True)

In [ ]:
# download the latest version of artifact raw_data.csv
artifact = run.use_artifact("churn_prediction_project_nn/raw_data.csv:latest")

# create a dataframe from the artifact
df = pd.read_csv(artifact.file())

#### 1.2.1 Pandas Profiling

In [ ]:
ProfileReport(df, title= "Pandas Profiling Report", explorative=True)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
run.finish()

### 1.3 Preprocessing

In [ ]:
input_artifact="churn_prediction_project_nn/raw_data.csv:latest"
artifact_name="preprocessed_data.csv"
artifact_type="clean_data"
artifact_description="Data after preprocessing"

In [ ]:
# create a new job_type
run = wandb.init(project="churn_prediction_project_nn", job_type="process_data")

In [ ]:
# download the latest version of artifact raw_data.csv
artifact=run.use_artifact(input_artifact)

# create a dataframe from the artifact
df = pd.read_csv(artifact.file())

In [ ]:
# delete duplicated rows
df.drop_duplicates(inplace=True)

# generate a "clean data file"
df.to_csv(artifact_name, index=False)

In [ ]:
# Create a new artifact and configure with the necessary arguments
artifact = wandb.Artifact(name=artifact_name,
                         type=artifact_type,
                         description=artifact_description)
artifact.add_file(artifact_name)

<ManifestEntry digest: 2VNPzyBON65Yp9cxPORlnA==>

In [ ]:
run.log_artifact(artifact)

In [ ]:
run.finish()

## 2 - Data Check

In [ ]:
%%file test_data.py
import pytest
import wandb
import pandas as pd

# This is global so all tests are collected under the same run
run = wandb.init(project="churn_prediction_project_nn", job_type="data_checks")

@pytest.fixture(scope="session")
def data():

    local_path = run.use_artifact("churn_prediction_project_nn/preprocessed_data.csv:latest").file()
    df = pd.read_csv(local_path)

    return df

def test_data_length(data):
    """
    We test that we have enough data to continue
    """
    assert len(data) > 1000


def test_number_of_columns(data):
    """
    We test that we have enough data to continue
    """
    assert data.shape[1] == 11

def test_column_presence_and_type(data):

    required_columns = {
        #"CustomerId": pd.api.types.is_int64_dtype,
        #"Surname": pd.api.types.is_object_dtype,
        "CreditScore": pd.api.types.is_int64_dtype,
        "Geography": pd.api.types.is_object_dtype,
        "Gender": pd.api.types.is_object_dtype,
        "Age": pd.api.types.is_int64_dtype,
        "Tenure": pd.api.types.is_int64_dtype,
        "Balance": pd.api.types.is_float_dtype,
        "NumOfProducts": pd.api.types.is_int64_dtype,
        "HasCrCard": pd.api.types.is_int64_dtype,
        "IsActiveMember": pd.api.types.is_int64_dtype,
        "EstimatedSalary": pd.api.types.is_float_dtype,  
        "Exited": pd.api.types.is_int64_dtype
    }

    # Check column presence
    assert set(data.columns.values).issuperset(set(required_columns.keys()))

    for col_name, format_verification_funct in required_columns.items():

        assert format_verification_funct(data[col_name]), f"Column {col_name} failed test {format_verification_funct}"


def test_class_names(data):

    # Check that only the known classes are present
    known_classes = [
        0,
        1
    ]

    assert data["Exited"].isin(known_classes).all()


def test_column_ranges(data):

    ranges = {
        "CreditScore": (0, 1000),
        "Age": (0,100),
        "Tenure": (0,10),
        "Balance": (0, 1.484705e+06),
        "NumOfProducts": (1,4),
        "HasCrCard": (0,1),
        "IsActiveMember": (0,1),
        "EstimatedSalary": (0, 1.484705e+06),
        "Exited": (0, 1)
    }

    for col_name, (minimum, maximum) in ranges.items():

        assert data[col_name].dropna().between(minimum, maximum).all(), (
            f"Column {col_name} failed the test. Should be between {minimum} and {maximum}, "
            f"instead min={data[col_name].min()} and max={data[col_name].max()}"
        )

Overwriting test_data.py


In [ ]:
!pytest . -vv

Test session starts (platform: linux, Python 3.7.13, pytest 3.6.4, pytest-sugar 0.9.5)
cachedir: .pytest_cache
rootdir: /content, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_data.py::test_data_length ✓                                 20% ██        
 test_data.py::test_number_of_columns ✓                           40% ████      
 test_data.py::test_column_presence_and_type ✓                    60% ██████    
 test_data.py::test_class_names ✓                                 80% ████████  
 test_data.py::test_column_ranges ✓                              100% ██████████

Results (3.51s):
       5 passed


In [ ]:
run.finish()

## 3 - Data Segregation

In [ ]:
# global variables

# ratio - 70% train / 30% test
test_size = 0.30

# seed used to reproduce purposes
seed = 42

# reference (column) to stratify the data
stratify = "Exited"

# name of the input artifact
artifact_input_name = "churn_prediction_project_nn/preprocessed_data.csv:latest"

# type of the artifact
artifact_type = "segregated_data"

In [ ]:
# configure logging 
logging.basicConfig(level=logging.INFO,
                   format="%(asctime)s %(message)s",
                   datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging object
logger = logging.getLogger()

# init wandb project
run = wandb.init(project="churn_prediction_project_nn", job_type="split_data")

logger.info("Downloading and reading artifact")
artifact=run.use_artifact(artifact_input_name)
artifact_path=artifact.file()
df = pd.read_csv(artifact_path)

# Split in train/test
logger.info("Splitting data into train and test")
splits = {}

splits["train"], splits["test"] = train_test_split(df,
                                                  test_size=test_size,
                                                  random_state=seed,
                                                  stratify=df[stratify])



14-07-2022 12:15:49 Downloading and reading artifact
14-07-2022 12:15:50 Splitting data into train and test


In [ ]:
# Save artifacts
with tempfile.TemporaryDirectory() as tmp_dir:
    
    for split, df in splits.items():
        
        # Make the artifact name from the name of the split plus the provided root
        artifact_name = f"{split}.csv"
        
        # Get the path on disk within the temp directory
        temp_path = os.path.join(tmp_dir,artifact_name)
        
        logger.info(f"Uploading the {split} dataset to {artifact_name}")
        
        # Save then upload to W&B
        df.to_csv(temp_path,index=False)
        
        artifact = wandb.Artifact(name=artifact_name,
                                 type=artifact_type,
                                 description=f"{split} split of dataset {artifact_input_name}")
        artifact.add_file(temp_path)
        
        logger.info("Logging artifact")
        run.log_artifact(artifact)
        
        artifact.wait()

14-07-2022 12:15:50 Uploading the train dataset to train.csv
14-07-2022 12:15:50 Logging artifact
14-07-2022 12:15:53 Uploading the test dataset to test.csv
14-07-2022 12:15:53 Logging artifact


In [ ]:
run.finish()

## 4 - Trainning

### 4.1 Holdout Configuration

In [ ]:
# global variables

# ratio used to split train and validation data
val_size = 0.30

# seed used to reproduce purposes
seed = 42

# reference (column) to stratify the data
stratify = "Exited"

# name of the input artifact
artifact_input_name = "churn_prediction_project_nn/train.csv:latest"

#entity
entity_name = "eec1509"

# project name
project_name = "churn_prediction_project_nn"

# type of the artifact
artifact_type = "Train"

In [ ]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

# initiate the wandb project
run = wandb.init(project="churn_prediction_project_nn", job_type="train")

logger.info("Downloading and reading train artifact")
local_path = run.use_artifact(artifact_input_name).file()
df_train = pd.read_csv(local_path)

# Spliting train.csv into train and validation dataset
logger.info("Spliting data into train/val")
# split-out train/validation and test dataset
x_train, x_val, y_train, y_val = train_test_split(df_train.drop(labels=stratify,axis=1),
                                                  df_train[stratify],
                                                  test_size=val_size,
                                                  random_state=seed,
                                                  shuffle=True,
                                                  stratify=df_train[stratify])

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


binary_accuracy,▁▆▇▇▇▇▇▇██▇█▇▇▇▇▇█▇███▇██▇▇▇█▇██████▇██▇
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1_score,▁▁▅▄▆▆▆▇█▆▆▆▇▆▇▆▇█▇▇▇▇▇▇█▇▇█▇▆▆▇██▇▇▆▇▇▆
loss,█▄▃▂▂▂▂▂▂▁▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▂
val_binary_accuracy,▁▄▁▃▄▅▅▇▇███▇█▇█▆▇▆█▇▇█▆▇▇▇▆▇█▇▇█▇▆▇▆▇▇█
val_f1_score,▂▆▁▃▄▅▅▇▇███▇█▇▇▆▇▆▇▇▇▇▆▇▆▆▆▆▇▇▇▇▇▆▆▆▇▆▇
val_loss,▇▄██▄▅▂▁▂▁▁▁▁▁▁▁▂▁▂▁▂▁▁▂▂▂▂▂▂▁▂▁▁▂▂▂▂▁▂▂
best_epoch,25
best_val_binary_accuracy,0.86429
binary_accuracy,0.82753
epoch,100


21-07-2022 19:39:31 Downloading and reading train artifact
21-07-2022 19:39:31 Spliting data into train/val


In [ ]:
logger.info("x train: {}".format(x_train.shape))
logger.info("y train: {}".format(y_train.shape))
logger.info("x val: {}".format(x_val.shape))
logger.info("y val: {}".format(y_val.shape))

21-07-2022 19:39:31 x train: (4900, 10)
21-07-2022 19:39:31 y train: (4900,)
21-07-2022 19:39:31 x val: (2100, 10)
21-07-2022 19:39:31 y val: (2100,)


### 4.2 Data Preparation


#### 4.2.1 Outlier Removal

In [ ]:
logger.info("Outlier Removal")
# temporary variable
x = x_train.select_dtypes("float64").copy()

# identify outlier in the dataset
lof = LocalOutlierFactor()
outlier = lof.fit_predict(x)
mask = outlier != -1

21-07-2022 19:39:31 Outlier Removal


In [ ]:
logger.info("x_train shape [original]: {}".format(x_train.shape))
logger.info("x_train shape [outlier removal]: {}".format(x_train.loc[mask,:].shape))

21-07-2022 19:39:31 x_train shape [original]: (4900, 10)
21-07-2022 19:39:31 x_train shape [outlier removal]: (4882, 10)


In [ ]:
logger.info("y_train shape [original]: {}".format(y_train.shape))
logger.info("y_train shape [outlier removal]: {}".format(y_train.loc[mask].shape))

21-07-2022 19:39:31 y_train shape [original]: (4900,)
21-07-2022 19:39:31 y_train shape [outlier removal]: (4882,)


In [ ]:
# AVOID data leakage and you should not do this procedure in the preprocessing stage
# Note that we did not perform this procedure in the validation set
x_train = x_train.loc[mask,:].copy()
y_train = y_train[mask].copy()

#### 4.2.2 Data Balancing

In [ ]:
# Shows the amount of values in the target column
y_train.value_counts()

0    3890
1     992
Name: Exited, dtype: int64

In [ ]:
# Select Categorical Indices
xx = x_train.iloc[1,:].values
xxx=pd.DataFrame([xx])
cols = xxx.columns
num_cols = xxx._get_numeric_data().columns
cat_indices = list(set(cols) - set(num_cols))

In [ ]:
from imblearn.over_sampling import SMOTENC
oversample = SMOTENC(categorical_features=cat_indices,  sampling_strategy = 0.6, random_state=seed) 
#x_train, y_train = oversample.fit_resample(x_train, y_train)

In [ ]:
y_train.value_counts()

0    3890
1     992
Name: Exited, dtype: int64

#### 4.2.3 Target Variable Encoding

In this case, the target variable is already encoded, but let's create an encoder to transform the numeric variable into categorical.

In [ ]:
logger.info("Encoding a Target Variable")

# define a categorical encoding for target variable
le = LabelEncoder()
le.fit(["Contiuned", "Exited"])
teste = le.inverse_transform(y_train)

21-07-2022 19:39:31 Encoding a Target Variable


### 4.3 Creating a Data Transform Pipeline

#### 4.3.1 Transformers

In [ ]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    # Class Constructor
    def __init__(self, feature_names):
        self.feature_names = feature_names
        
    # Return self nothing to do here
    def fit(self, X, y=None):
        return self
    
    # Method that describes what tis custom transformer need to do
    def transform(self, X, y=None):
        return X[self.feature_names]
    
# Handling categorical features
class CategoricalTransformer(BaseEstimator, TransformerMixin):
    # Class constructor method that takes one boolean as argument
    def __init__(self, new_features=True, colnames=None):
        self.new_features = new_features
        self.colnames = colnames
        
    # Return self nothing else to do here
    def fit(self, X, y=None):
        return self
    
    def get_feature_names_out(self):
        return self.colnames.tolist()
    
    # Transformer method we wrote for this transformer
    def transform(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)
        
        # Remove white space in categorical features
        df = df.apply(lambda row: row.str.strip())
        
        # customize feature?
        # How can I identify what needs to be modified? EDA!!!!!!
        if self.new_features:
            
            # replace ? with unknown
            edit_cols = ['Geography', 'Gender']
            for col in edit_cols:
                df.loc[df[col].str.contains("\?"), col] = 'unknown'
        
        # update column names
        self.colnames = df.columns
        df = pd.DataFrame(X, columns=self.colnames)
        
        return df

# transform numerical features
class NumericalTransformer(BaseEstimator, TransformerMixin):
    # Class constructor method that takes a model parameter as its argument
    # model 0: minmax
    # model 1: standard
    # model 2: without scaler
    def __init__(self, model=0, colnames=None):
        self.model = model
        self.colnames = colnames
        self.scaler = None

    # Fit is used only to learn statistical about Scalers
    def fit(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)
        
        # minmax
        if self.model == 0:
            self.scaler = MinMaxScaler()
            self.scaler.fit(df)
        # standard scaler
        elif self.model == 1:
            self.scaler = StandardScaler()
            self.scaler.fit(df)
        return self

    # return columns names after transformation
    def get_feature_names_out(self):
        return self.colnames

    # Transformer method we wrote for this transformer
    # Use fitted scalers
    def transform(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)
        
        # update columns name
        self.colnames = df.columns.tolist()

        # minmax
        if self.model == 0:
            # transform data
            df = self.scaler.transform(df)
        elif self.model == 1:
            # transform data
            df = self.scaler.transform(df)
        else:
            df = df.values

        return df

#### 4.3.2 Holdout Pipeline

In [ ]:
# model = 0 (min-max), 1 (z-score), 2 (without normalization)
numerical_model = 1

# Categrical features to pass down the categorical pipeline
categorical_features = x_train.select_dtypes("object").columns.to_list()

# Numerical features to pass down the numerical pipeline
numerical_features = x_train.select_dtypes(["int64","float"]).columns.to_list()

# Defining the steps for the categorical pipeline
categorical_pipeline = Pipeline(steps=[('cat_selector', FeatureSelector(categorical_features)),
                                       ('imputer_cat', SimpleImputer(strategy="most_frequent")),
                                       ('cat_transformer', CategoricalTransformer(colnames=categorical_features)),
                                       ('cat_encoder', OneHotEncoder(sparse=False, drop="first"))])

# Defining the steps in the numerical pipeline
numerical_pipeline = Pipeline(steps=[('num_selector', FeatureSelector(numerical_features)),
                                     ('imputer_num', SimpleImputer(strategy="median")),
                                     ('num_transformer', NumericalTransformer(numerical_model, 
                                                                              colnames=numerical_features))])

# Combine numerical and categorical pieplines into one full big pipeline horizontally
pipe = FeatureUnion(transformer_list=[('cat_pipeline', categorical_pipeline),
                                                             ('num_pipeline', numerical_pipeline)])

#### 4.3.3 Transforming

In [ ]:
# Transforming
logger.info("Transforming")
x_train_trans = pipe.fit_transform(x_train)
x_val_trans = pipe.transform(x_val)

21-07-2022 19:39:42 Transforming


In [ ]:
# Convert format to pandas data frame
# cat_names is a numpy array
cat_names = pipe.get_params()["cat_pipeline"][3].get_feature_names_out().tolist()
# num_names is a list
num_names = pipe.get_params()["num_pipeline"][2].get_feature_names_out()
x_train_df = pd.DataFrame(x_train_trans,columns = cat_names + num_names)
x_val_df = pd.DataFrame(x_val_trans,columns = cat_names + num_names)
# Convert pandas series to pandas data frame
y_train_df = pd.DataFrame(y_train)
y_val_df = pd.DataFrame(y_val)

#### 4.3.4 Export Transformed Data

In [ ]:
run = wandb.init(project="churn_prediction_project_nn", job_type="train")

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


In [ ]:
# Concat x and y
train_df = pd.concat([x_train_df.reset_index(drop=True), y_train_df.reset_index(drop=True)], axis=1) 
val_df = pd.concat([x_val_df.reset_index(drop=True), y_val_df.reset_index(drop=True)], axis=1)

In [ ]:
# saving transformed train data to csv
train_df.to_csv("transformed_train_data.csv", index=False)
# Saving artifact to wandb
!wandb artifact put \
       --name churn_prediction_project_nn/transformed_train_data.csv \
       --type transformed_data \
       --description "Upload Transformed Train Data" transformed_train_data.csv

wandb: Uploading file transformed_train_data.csv to: "eec1509/churn_prediction_project_nn/transformed_train_data.csv:latest" (transformed_data)
wandb: Currently logged in as: macleal (eec1509). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.12.21
wandb: Run data is saved locally in /content/wandb/run-20220721_193953-2o8erxsx
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run breezy-firebrand-510
wandb: ⭐️ View project at https://wandb.ai/eec1509/churn_prediction_project_nn
wandb: 🚀 View run at https://wandb.ai/eec1509/churn_prediction_project_nn/runs/2o8erxsx
Artifact uploaded, use this artifact in a run by adding:

    artifact = run.use_artifact("eec1509/churn_prediction_project_nn/transformed_train_data.csv:v6")

wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: Synced breezy-firebrand-510: https://wandb.ai/eec1509/churn_prediction

In [ ]:
# saving transformed val data to csv
val_df.to_csv("transformed_val_data.csv", index=False)
# Saving artifact to wandb
!wandb artifact put \
       --name churn_prediction_project_nn/transformed_val_data.csv \
       --type transformed_data \
       --description "Upload Transformed Validation Data" transformed_val_data.csv

wandb: Uploading file transformed_val_data.csv to: "eec1509/churn_prediction_project_nn/transformed_val_data.csv:latest" (transformed_data)
wandb: Currently logged in as: macleal (eec1509). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.12.21
wandb: Run data is saved locally in /content/wandb/run-20220721_194001-1q8ekxns
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run scarlet-snowball-511
wandb: ⭐️ View project at https://wandb.ai/eec1509/churn_prediction_project_nn
wandb: 🚀 View run at https://wandb.ai/eec1509/churn_prediction_project_nn/runs/1q8ekxns
Artifact uploaded, use this artifact in a run by adding:

    artifact = run.use_artifact("eec1509/churn_prediction_project_nn/transformed_val_data.csv:v5")

wandb: Waiting for W&B process to finish... (success).
wandb:                                                                                
wandb: Synced scarlet-snowball-511: https://wandb.ai/eec1509/churn_prediction_proje

In [ ]:
run.finish()

### 4.4 Hyperparameter Tuning

#### 4.4.1 Import Transformed Data

In [ ]:
# configure logging 
logging.basicConfig(level=logging.INFO,
                   format="%(asctime)s %(message)s",
                   datefmt='%d-%m-%Y %H:%M:%S')

# reference for a logging object
logger = logging.getLogger()

#entity
entity_name = "eec1509"
# project name
project_name = "churn_prediction_project_nn"

In [ ]:
# init wandb project
logger = logging.getLogger()
run = wandb.init(project="churn_prediction_project_nn", job_type="train")

logger.info("Downloading and reading transformed train data artifact")
artifact=run.use_artifact("churn_prediction_project_nn/transformed_train_data.csv:latest")
artifact_path=artifact.file()
train = pd.read_csv(artifact_path)

logger.info("Downloading and reading transformed validation data artifact")
artifact=run.use_artifact("churn_prediction_project_nn/transformed_val_data.csv:latest")
artifact_path=artifact.file()
val = pd.read_csv(artifact_path)

wandb: Currently logged in as: macleal (eec1509). Use `wandb login --relogin` to force relogin


22-07-2022 15:31:02 Downloading and reading transformed train data artifact
22-07-2022 15:31:03 Downloading and reading transformed validation data artifact


In [ ]:
# seperate data as idepedent variables and dependent variable
# Extract the target from the features
logger.info("Extracting target from dataframe")
x_train = train.copy()
y_train = x_train.pop("Exited")

22-07-2022 15:31:03 Extracting target from dataframe


In [ ]:
# seperate data as idepedent variables and dependent variable
# Extract the target from the features
logger.info("Extracting target from dataframe")
x_val = val.copy()
y_val = x_val.pop("Exited")

22-07-2022 15:31:03 Extracting target from dataframe


#### 4.4.2 Monitoring a neural network

In [ ]:
# Default values for hyperparameters
defaults = dict(
            layer_1 = 15,
            layer_2 = 8,
            n_noise = 0.4,
            learn_rate = 0.005,
            L2_rate_1 = 0.3,
            L2_rate_2 = 0.2,
            batch_size = 512,
            dropout_1 = 0.13,
            dropout_2 = 0.22,
            n_patience = 200,
            optim = "Adam_clip",
            grad_clip = 0.2,
            # Technics Activation
            n_noise_on = 1,
            L2_rate_1_on = 0,
            L2_rate_2_on = 0,
            bath_norm_on = 1,
            dropout_1_on = 0,
            dropout_2_on = 1,
            n_epochs = 500)

# Initialize a new wandb run
wandb.init(project=project_name, config= defaults)

# Config is a variable that holds and saves hyperparameters and inputs
config = wandb.config

In [ ]:
# Instantiate a simple classification model
model = tf.keras.Sequential()
# First Layer
if config.n_noise_on==1:
  model.add(tf.keras.layers.GaussianNoise(config.n_noise))

if config.L2_rate_1_on==1:
  model.add(Dense(config.layer_1, activation=tf.nn.relu,
                kernel_regularizer = tf.keras.regularizers.L2(l2=config.L2_rate_1),
                kernel_initializer='he_uniform'))
else:
  model.add(Dense(config.layer_1, activation=tf.nn.relu,
                kernel_initializer='he_uniform'))
  
if config.bath_norm_on==1:
  model.add(layers.BatchNormalization())

if config.dropout_1_on==1 and config.bath_norm_on!=1:
  model.add(layers.Dropout(config.dropout_1))

# Second Layer
if config.n_noise_on==1:
  model.add(tf.keras.layers.GaussianNoise(config.n_noise))

if config.L2_rate_2_on==1:
  model.add(Dense(config.layer_2, activation=tf.nn.relu,
                kernel_regularizer = tf.keras.regularizers.L2(l2=config.L2_rate_2),
                kernel_initializer='he_uniform'))
else:
  model.add(Dense(config.layer_2, activation=tf.nn.relu,
                kernel_initializer='he_uniform'))
  
if config.bath_norm_on==1:
  model.add(layers.BatchNormalization())

if config.dropout_2_on==1 and config.bath_norm_on!=1:
  model.add(layers.Dropout(config.dropout_2))
  
# Exit Layer
model.add(Dense(1, activation=tf.nn.sigmoid))

# Instantiate a logistic loss function that expects integer targets.
loss = tf.keras.losses.BinaryCrossentropy()

# Instantiate an metrics.
accuracy = tf.keras.metrics.BinaryAccuracy()
f1Score = tfa.metrics.F1Score(num_classes=1, threshold=0.5)

# Instantiate an optimizer.
if config.optim=="SGD":
  optimizer = tf.keras.optimizers.SGD(learning_rate=config.learn_rate)
elif config.optim=="SGD_clip":
  optimizer = tf.keras.optimizers.SGD(learning_rate=config.learn_rate, clipvalue = config.grad_clip)
elif config.optim=="Adam":
  optimizer = tf.keras.optimizers.Adam(learning_rate=config.learn_rate)
else:
  optimizer = tf.keras.optimizers.Adam(learning_rate=config.learn_rate, clipvalue = config.grad_clip)

# configure the optimizer, loss, and metrics to monitor.
model.compile(optimizer=optimizer, loss=loss, metrics=[accuracy, f1Score])  

In [ ]:
%%wandb
# Add WandbCallback() to the fit function
hist = model.fit(x_train, y_train, batch_size=config.batch_size,
          epochs=config.n_epochs,
          validation_data=(x_val, y_val),
          callbacks=[WandbCallback(monitor='val_binary_accuracy', mode='max', log_weights=True), 
          EarlyStopping(monitor='val_binary_accuracy', mode='max', patience=config.n_patience)],
          verbose=0)   

wandb: WARNING Unable to compute FLOPs for this model.


In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.legend(['Train', 'Val'], loc = 'upper left')
plt.show()

#### 4.4.3 Sweeps

* **Metodologia dos sweeps:** Foram escolhidas três abordagens que utilizam diferentes técnicas de generalização (Classical, Alternate e Modern) e as varreduras tem como objetivo analisar qual estratégia tem melhor desempenho com o dataset escolhido além de realizar o ajuste dos hiperparâmetros relacionados com cada abordagem visando a obtenção dos melhores resultados.
  * Classical Regularization: Early Stopping + L2 Regularization;
    * Pode fazer uso do Batch Normalization.
  * Alternate Regularization: Early Stopping + Noise + Weight Constraint;
    * Pode fazer uso do Batch Normalization.
  * Modern Regularization: Early Stopping + Dropout + Weight Constraint.
    * Não se pode utilizar o Batch Normalization devido ao uso do Dropout.



In [ ]:
# The sweep calls this function with each set of hyperparameters
def train():
    # Default values for hyper-parameters we're going to sweep over
    defaults = dict(regularization_method = 'Classical',
                    use_bnorm = True,
                    nodes_layer1 = 8,
                    nodes_layer2 = 8,
                    L2_rate1 = 0.5,
                    L2_rate2 = 0.5,
                    n_noise1 = 0.5,
                    n_noise2 = 0.5,
                    n_noise3 = 0.5,
                    min_norm_value1 = 0,
                    min_norm_value2 = 0,
                    max_norm_value1 = 1,
                    max_norm_value2 = 1,
                    n_dropout1 = 0.5,
                    n_dropout2 = 0.5,
                    use_optimizer = 'SGD',
                    Learning_rate = 0.1,
                    clip_value = 1,
                    batch_size = 100,
                    n_epochs = 100,
                    n_patience = 50)
    
    # Initialize a new wandb run
    wandb.init(config=defaults)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    model = tf.keras.Sequential()
    # Classical Regularization: Early Stopping + L2 Regularization
    if config.regularization_method == 'Classical':
      if config.use_bnorm == True:
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(config.nodes_layer1, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.L2(l2=config.L2_rate1)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(config.nodes_layer2, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.L2(l2=config.L2_rate2)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))
      else:
        model.add(tf.keras.layers.Dense(config.nodes_layer1, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.L2(l2=config.L2_rate1)))
        model.add(tf.keras.layers.Dense(config.nodes_layer2, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.L2(l2=config.L2_rate2)))
        model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

    # Alternate Regularization: Early Stopping + Noise + Weight Constraint
    if config.regularization_method == 'Alternate':
      if config.use_bnorm == True:        
        model.add(tf.keras.layers.GaussianNoise(config.n_noise1))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(config.nodes_layer1, activation=tf.nn.relu, kernel_constraint=tf.keras.constraints.MinMaxNorm(min_value=config.min_norm_value1, max_value=config.max_norm_value1)))
        model.add(tf.keras.layers.GaussianNoise(config.n_noise2))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(config.nodes_layer2, activation=tf.nn.relu, kernel_constraint=tf.keras.constraints.MinMaxNorm(min_value=config.min_norm_value2, max_value=config.max_norm_value2)))
        model.add(tf.keras.layers.GaussianNoise(config.n_noise3))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))
      else:
        model.add(tf.keras.layers.GaussianNoise(config.n_noise1))
        model.add(tf.keras.layers.Dense(config.nodes_layer1, activation=tf.nn.relu, kernel_constraint=tf.keras.constraints.MinMaxNorm(min_value=config.min_norm_value1, max_value=config.max_norm_value2)))
        model.add(tf.keras.layers.GaussianNoise(config.n_noise2))
        model.add(tf.keras.layers.Dense(config.nodes_layer2, activation=tf.nn.relu, kernel_constraint=tf.keras.constraints.MinMaxNorm(min_value=config.min_norm_value1, max_value=config.max_norm_value2)))
        model.add(tf.keras.layers.GaussianNoise(config.n_noise3))
        model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))
    # Modern Regularization: Early Stopping + Dropout + Weight Constraint
    if config.regularization_method == 'Modern':
      model.add(tf.keras.layers.Dense(config.nodes_layer1, activation=tf.nn.relu, kernel_constraint=tf.keras.constraints.MinMaxNorm(min_value=config.min_norm_value1, max_value=config.max_norm_value2)))
      model.add(tf.keras.layers.Dropout(config.n_dropout1))
      model.add(tf.keras.layers.Dense(config.nodes_layer2, activation=tf.nn.relu, kernel_constraint=tf.keras.constraints.MinMaxNorm(min_value=config.min_norm_value1, max_value=config.max_norm_value2)))
      model.add(tf.keras.layers.Dropout(config.n_dropout2))
      model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

    # Instantiate a logistic loss function that expects integer targets.
    loss = tf.keras.losses.BinaryCrossentropy()

    # Instantiate an metrics.
    accuracy = tf.keras.metrics.BinaryAccuracy()

    # Instantiate an optimizer.
    if config.use_optimizer == 'SGD':
      optimizer = tf.keras.optimizers.SGD(learning_rate=config.Learning_rate, clipvalue=config.clip_value)
    elif config.use_optimizer == 'Adam':
      optimizer = tf.keras.optimizers.Adam(learning_rate=config.Learning_rate, clipvalue=config.clip_value)

    # configure the optimizer, loss, and metrics to monitor.
    model.compile(optimizer=optimizer, loss=loss, metrics=[accuracy])

    model.fit(x_train,
              y_train,
              batch_size=config.batch_size,
              epochs=config.n_epochs,
              validation_data=(x_val, y_val),
              callbacks=[WandbCallback(monitor='val_binary_accuracy', mode='max', log_weights=True), 
              EarlyStopping(monitor='val_binary_accuracy', mode='auto', patience=config.n_patience)],
              verbose=0)   

In [ ]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_binary_accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'regularization_method': {
            'values': ['Classical', 'Alternate', 'Modern']
        },
        'use_bnorm': {
            'values': [True, False]  
        },
        'nodes_layer1': {
            'max': 100,
            'min': 4,
            'distribution': 'int_uniform',
        },
        'nodes_layer2': {
            'max': 100,
            'min': 4,
            'distribution': 'int_uniform',
        },
        'L2_rate1': {
            'distribution': 'normal',
            'mu': 0.5,
            'sigma': 0.15, 
        },
        'L2_rate2': {
            'distribution': 'normal',
            'mu': 0.5,
            'sigma': 0.15, 
        },        
        'n_noise1': {
            'distribution': 'normal',
            'mu': 0.5,
            'sigma': 0.05, 
        },        
        'n_noise2': {
            'distribution': 'normal',
            'mu': 0.5,
            'sigma': 0.05, 
        },        
        'n_noise3': {
            'distribution': 'normal',
            'mu': 0.5,
            'sigma': 0.05, 
        },        
        'min_norm_value1': {
            'distribution': 'normal',
            'mu': 0.3,
            'sigma': 0.1, 
        },        
        'min_norm_value2': {
            'distribution': 'normal',
            'mu': 0.3,
            'sigma': 0.1, 
        },        
        'max_norm_value1': {
            'distribution': 'normal',
            'mu': 0.7,
            'sigma': 0.1, 
        },        
        'max_norm_value2': {
            'distribution': 'normal',
            'mu': 0.7,
            'sigma': 0.1, 
        },
        'n_dropout1': {
            'distribution': 'normal',
            'mu': 0.5,
            'sigma': 0.05, 
        },
        'n_dropout2': {
            'distribution': 'normal',
            'mu': 0.5,
            'sigma': 0.05, 
        },
        'use_optimizer': {
            'values': ['SGD', 'Adam']
        },
        'Learning_rate': {
            'min': -4,
            'max': -2,
            'distribution': 'log_uniform',  
        },
        'clip_value': {
            'distribution': 'normal',
            'mu': 0.3,
            'sigma': 0.1, 
        },
        'batch_size': {
            'values': [64, 128, 256]
        },
        'n_epochs': {
            'values': [500, 1000, 2000]
        },
        'n_patience': {
            'values': [50, 75, 100]
        },
    }
}

In [ ]:
# Initialize a new sweep
# Arguments:
#     – sweep_config: the sweep config dictionary defined above
#     – entity: Set the username for the sweep
#     – project: Set the project name for the sweep
sweep_id = wandb.sweep(sweep_config, entity=entity_name, project=project_name)

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Learning_rate uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: 8iw0rtj2
Sweep URL: https://wandb.ai/eec1509/churn_prediction_project_nn/sweeps/8iw0rtj2


In [ ]:
# Initialize a new sweep
# Arguments:
#     – sweep_id: the sweep_id to run - this was returned above by wandb.sweep()
#     – function: function that defines your model architecture and trains it
wandb.agent(sweep_id = sweep_id, function=train,count=20)

wandb: Agent Starting Run: 7bdfvlgu with config:
wandb: 	L2_rate1: 0.5512982412012013
wandb: 	L2_rate2: 0.4060785769432202
wandb: 	Learning_rate: 0.06382337394227326
wandb: 	batch_size: 128
wandb: 	clip_value: 0.19707804619469577
wandb: 	max_norm_value1: 0.8385076138446061
wandb: 	max_norm_value2: 0.6391822881263921
wandb: 	min_norm_value1: 0.21558363368754452
wandb: 	min_norm_value2: 0.3061945619744403
wandb: 	n_dropout1: 0.4806802499795397
wandb: 	n_dropout2: 0.45638372137038546
wandb: 	n_epochs: 500
wandb: 	n_noise1: 0.4543018232022832
wandb: 	n_noise2: 0.5130711730787214
wandb: 	n_noise3: 0.5068508726158102
wandb: 	n_patience: 75
wandb: 	nodes_layer1: 68
wandb: 	nodes_layer2: 84
wandb: 	regularization_method: Alternate
wandb: 	use_bnorm: False
wandb: 	use_optimizer: SGD


wandb: WARNING Unable to compute FLOPs for this model.


binary_accuracy,▁▃▂▃▃▄▄▅▄▅▆▆▅▇▆▇▆▆▆▇▇▇▇▆▇▇█▇▇▆█▇█▇▇▇▇██▇
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▅▅▄▄▄▃▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▂▁▁▁▂▂▁▁▁▂▁
val_binary_accuracy,▁▁▁▂▂▃▃▄▄▄▆▄▄▇▇▇█▇▆█▇▆▇▇██▇▇▇▆▇█▇▇▇▇▇███
val_loss,█▇▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
best_epoch,297
best_val_loss,0.36246
binary_accuracy,0.82528
epoch,297
loss,0.4042
val_binary_accuracy,0.85714


wandb: Agent Starting Run: m2i6c0zd with config:
wandb: 	L2_rate1: 0.27107254339886333
wandb: 	L2_rate2: 0.39746596128688416
wandb: 	Learning_rate: 0.040179648287535195
wandb: 	batch_size: 256
wandb: 	clip_value: 0.2399377318648322
wandb: 	max_norm_value1: 0.6636060662070615
wandb: 	max_norm_value2: 0.8452061540819615
wandb: 	min_norm_value1: 0.3149212082254995
wandb: 	min_norm_value2: 0.1663160243011807
wandb: 	n_dropout1: 0.45343720483246774
wandb: 	n_dropout2: 0.40673133589603694
wandb: 	n_epochs: 500
wandb: 	n_noise1: 0.5557697780298455
wandb: 	n_noise2: 0.43590745515802914
wandb: 	n_noise3: 0.5175224684030405
wandb: 	n_patience: 50
wandb: 	nodes_layer1: 14
wandb: 	nodes_layer2: 13
wandb: 	regularization_method: Alternate
wandb: 	use_bnorm: True
wandb: 	use_optimizer: Adam


wandb: WARNING Unable to compute FLOPs for this model.


...

wandb: WARNING Unable to compute FLOPs for this model.


binary_accuracy,▁▁▂▂▃▄▄▅▅▆▆▆▆▆▆▆▆▇▇▇▇▆▆▆▇▇▇▆▇▇▇▇▇▇▇█▇▇▇█
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁
val_binary_accuracy,▁▁▁▁▃▅▅▆▇▇▇▇▇▇▇▇█▇█▇█▇█▇██▇██████▇██████
val_loss,█▇▆▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,248
best_val_loss,0.33914
binary_accuracy,0.84576
epoch,256
loss,0.37255
val_binary_accuracy,0.86238


wandb: Agent Starting Run: 8c0x0nfk with config:
wandb: 	L2_rate1: 0.6215941176026736
wandb: 	L2_rate2: 0.18636309041305937
wandb: 	Learning_rate: 0.02597847391776992
wandb: 	batch_size: 64
wandb: 	clip_value: 0.3237933648722596
wandb: 	max_norm_value1: 0.6743372268278462
wandb: 	max_norm_value2: 0.8464002072509957
wandb: 	min_norm_value1: 0.2532738163199169
wandb: 	min_norm_value2: 0.3018827460304915
wandb: 	n_dropout1: 0.4099834242188865
wandb: 	n_dropout2: 0.4632669964169688
wandb: 	n_epochs: 1000
wandb: 	n_noise1: 0.5623533177553722
wandb: 	n_noise2: 0.5120483262565183
wandb: 	n_noise3: 0.4905704883114588
wandb: 	n_patience: 75
wandb: 	nodes_layer1: 91
wandb: 	nodes_layer2: 72
wandb: 	regularization_method: Alternate
wandb: 	use_bnorm: True
wandb: 	use_optimizer: SGD


wandb: WARNING Unable to compute FLOPs for this model.


binary_accuracy,▁▃▃▅▆▅▆▇▆▇▆▇▇▇▇▆▇▇▇▆▇█▇▆█▇▇▇▇▇▇▇▇▇▇█▇▇█▇
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▅▄▃▃▃▂▂▃▃▂▂▂▂▂▂▂▂▂▁▁▂▂▁▂▂▂▂▁▁▂▂▂▁▁▁▁▁▁
val_binary_accuracy,▁▂▂▄▄▅▅▅▆▅▆▇▇▆▇▇▇▇████▇▇▇███████████████
val_loss,█▆▅▄▄▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁
best_epoch,333
best_val_loss,0.35278
binary_accuracy,0.82487
epoch,348
loss,0.41356
val_binary_accuracy,0.8581


---

In [ ]:
run.finish()

### 4.5 Export the best model

In [ ]:
run = wandb.init(entity="eec1509", project="churn_prediction_modern_nn", job_type="best_model")

#### 4.5.1 Import the best wandb sweep

In [ ]:
# restore the raw model file (Insert the wandb best sweep path)
best_model_path = "eec1509/churn_prediction_modern_nn/j2kzy0k9"
best_model = wandb.restore('model-best.h5', run_path=best_model_path)

# restore the model for tf.keras
model = tf.keras.models.load_model(best_model.name)

#### 4.5.2 Export Encoders and Best Model

In [ ]:
# Configure logging
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(message)s", datefmt='%d-%m-%Y %H:%M:%S')
# types and names of the artifacts
artifact_type = "inference_artifact"
artifact_transform = "data_transform"
artifact_encoder = "target_encoder"
artifact_model = "model_export"

In [ ]:
logger.info("Dumping the artifacts to disk")

In [ ]:
# Export Model artifact
artifact = wandb.Artifact(artifact_model,
                          type=artifact_type,
                          description="Neural Network Model for Classification Purpose"
                          )

logger.info("Logging model artifact")
model.save("path")
artifact.add_dir("path")
run.log_artifact(artifact)

25-07-2022 15:04:22 Logging model artifact


INFO:tensorflow:Assets written to: path/assets


25-07-2022 15:04:24 Assets written to: path/assets
wandb: Adding directory to artifact (./path)... Done. 0.1s


In [ ]:
# Export the pipe data transform using joblib
joblib.dump(pipe, artifact_transform)

# Pipe Data Transform
artifact = wandb.Artifact(artifact_transform,
                          type=artifact_type,
                          description="Pipeline for Data Transform"
                          )

logger.info("Logging Pipeline for Data Transform")
artifact.add_file(artifact_transform)
run.log_artifact(artifact)

In [ ]:
# Export the target encoder using joblib
joblib.dump(le, artifact_encoder)

# Target encoder artifact
artifact = wandb.Artifact(artifact_encoder,
                          type=artifact_type,
                          description="The encoder used to encode the target variable"
                          )

logger.info("Logging target enconder artifact")
artifact.add_file(artifact_encoder)
run.log_artifact(artifact)

In [ ]:
run.finish()

## 5 - Testing

In [80]:
# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(message)s", datefmt='%d-%m-%Y %H:%M:%S')

# initiate the wandb project
run = wandb.init(project="churn_prediction_project_nn",job_type="test")

In [81]:
# Class for the data transform pipeline
class FeatureSelector(BaseEstimator, TransformerMixin):
    # Class Constructor
    def __init__(self, feature_names):
        self.feature_names = feature_names
        
    # Return self nothing to do here
    def fit(self, X, y=None):
        return self
    
    # Method that describes what tis custom transformer need to do
    def transform(self, X, y=None):
        return X[self.feature_names]
    
# Handling categorical features
class CategoricalTransformer(BaseEstimator, TransformerMixin):
    # Class constructor method that takes one boolean as argument
    def __init__(self, new_features=True, colnames=None):
        self.new_features = new_features
        self.colnames = colnames
        
    # Return self nothing else to do here
    def fit(self, X, y=None):
        return self
    
    def get_feature_names_out(self):
        return self.colnames.tolist()
    
    # Transformer method we wrote for this transformer
    def transform(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)
        
        # Remove white space in categorical features
        df = df.apply(lambda row: row.str.strip())
        
        # customize feature?
        # How can I identify what needs to be modified? EDA!!!!!!
        if self.new_features:
            
            # replace ? with unknown
            edit_cols = ['Geography', 'Gender']
            for col in edit_cols:
                df.loc[df[col].str.contains("\?"), col] = 'unknown'
        
        # update column names
        self.colnames = df.columns
        df = pd.DataFrame(X, columns=self.colnames)
        
        return df

# transform numerical features
class NumericalTransformer(BaseEstimator, TransformerMixin):
    # Class constructor method that takes a model parameter as its argument
    # model 0: minmax
    # model 1: standard
    # model 2: without scaler
    def __init__(self, model=0, colnames=None):
        self.model = model
        self.colnames = colnames
        self.scaler = None

    # Fit is used only to learn statistical about Scalers
    def fit(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)
        
        # minmax
        if self.model == 0:
            self.scaler = MinMaxScaler()
            self.scaler.fit(df)
        # standard scaler
        elif self.model == 1:
            self.scaler = StandardScaler()
            self.scaler.fit(df)
        return self

    # return columns names after transformation
    def get_feature_names_out(self):
        return self.colnames

    # Transformer method we wrote for this transformer
    # Use fitted scalers
    def transform(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)
        
        # update columns name
        self.colnames = df.columns.tolist()

        # minmax
        if self.model == 0:
            # transform data
            df = self.scaler.transform(df)
        elif self.model == 1:
            # transform data
            df = self.scaler.transform(df)
        else:
            df = df.values

        return df

In [82]:
# global variables
# path of the artifact related to test dataset
artifact_test_path = "eec1509/churn_prediction_project_nn/test.csv:latest"
# path of the pipeline for data transform
artifact_transform_path = "eec1509/churn_prediction_project_nn/data_transform:latest"
# path of the model artifact
artifact_model_path_unbalanced = "eec1509/churn_prediction_modern_nn/model_export:latest"
# path of the model artifact
artifact_model_path_balanced = "eec1509/churn_prediction_alternateb_nn/model_export:latest"
# path of the target encoder artifact
artifact_encoder_path = "eec1509/churn_prediction_project_nn/target_encoder:latest"

In [83]:
# Init wandb project
logger = logging.getLogger()
run = wandb.init(entity="eec1509", project="churn_prediction_project_nn", job_type="test")

logger.info("Downloading and reading transformed test data artifact")
artifact_path=run.use_artifact(artifact_test_path).file()
#artifact_path=artifact.file()
test = pd.read_csv(artifact_path)

# Download the pipeline for dada transform 
logger.info("Downloading the dada transform pipeline")
data_transform_path = run.use_artifact(artifact_transform_path).file()
pipe = joblib.load(data_transform_path)

# Download the target variable encoder
logger.info("Extracting the encoding of the target variable")
encoder_path = run.use_artifact(artifact_encoder_path).file()
le = joblib.load(encoder_path)

# Download inference artifact
logger.info("Downloading and load the exported model")
# use the latest version of the model
model_at = run.use_artifact(artifact_model_path_unbalanced)
# download the directory in which the model is saved
model_dir= model_at.download()
print("model: ", model_dir)
model = keras.models.load_model(model_dir)

25-07-2022 17:14:20 Downloading and reading transformed test data artifact
25-07-2022 17:14:20 Downloading the dada transform pipeline
25-07-2022 17:14:21 Extracting the encoding of the target variable
25-07-2022 17:14:21 Downloading and load the exported model


model:  ./artifacts/model_export:v1


In [84]:
# seperate data as idepedent variables and dependent variable
# Extract the target from the features
logger.info("Extracting target from dataframe")
x_test = test.copy()
y_test = x_test.pop("Exited")

25-07-2022 17:14:30 Extracting target from dataframe


In [85]:
x_test = pipe.transform(x_test)

In [86]:
y_test_encoded = le.inverse_transform(y_test)
y_test_encoded

array(['Contiuned', 'Contiuned', 'Contiuned', ..., 'Contiuned', 'Exited',
       'Contiuned'], dtype='<U9')

In [87]:
# predict
logger.info("Infering")
predict = model.predict(x_test)

25-07-2022 17:14:35 Infering


In [88]:
predict_rounded = np.rint(predict)

In [89]:
# execute the loss and accuracy using the test dataset
loss_, acc_ = model.evaluate(x=x_test,y=y_test, batch_size=64)
print('Test loss: %.3f - acc: %.3f' % (loss_, acc_))

47/47 [==============================] - 0s 2ms/step - loss: 0.3299 - binary_accuracy: 0.8647
Test loss: 0.330 - acc: 0.865


In [90]:
# Evaluation Metrics
logger.info("Test Evaluation metrics")
fbeta = fbeta_score(y_test, predict_rounded, beta=1, zero_division=1)
precision = precision_score(y_test, predict_rounded, zero_division=1)
recall = recall_score(y_test, predict_rounded, zero_division=1)
acc = accuracy_score(y_test, predict_rounded)

logger.info("Test Accuracy: {}".format(acc))
logger.info("Test Precision: {}".format(precision))
logger.info("Test Recall: {}".format(recall))
logger.info("Test F1: {}".format(fbeta))

run.summary["Acc"] = acc
run.summary["Precision"] = precision
run.summary["Recall"] = recall
run.summary["F1"] = fbeta

25-07-2022 17:14:42 Test Evaluation metrics
25-07-2022 17:14:42 Test Accuracy: 0.8646666666666667
25-07-2022 17:14:42 Test Precision: 0.7855153203342619
25-07-2022 17:14:42 Test Recall: 0.46153846153846156
25-07-2022 17:14:42 Test F1: 0.5814432989690722


In [91]:
# Compare the accuracy, precision, recall with previous ones
print(classification_report(y_test,predict_rounded))

              precision    recall  f1-score   support

           0       0.88      0.97      0.92      2389
           1       0.79      0.46      0.58       611

    accuracy                           0.86      3000
   macro avg       0.83      0.71      0.75      3000
weighted avg       0.86      0.86      0.85      3000



In [92]:
run.finish()

Acc,0.86467
F1,0.58144
Precision,0.78552
Recall,0.46154
